# Anon's MusicGen Remix
Welcome to my MusicGen remix demo jupyter notebook. I've stripped out everything unrelated to remixing songs using a guiding melody with the 'musicgen-melody' model to help streamline things. 

If you want to use another model for music generation continuation, use one of the other notebooks.

`facebook/musicgen-melody` - 1.5B transformer decoder also supporting melody conditioning.

In [ ]:
from audiocraft.models import MusicGen
from audiocraft.models import MultiBandDiffusion

USE_DIFFUSION_DECODER = False
# Using small model, better results would be obtained with `medium` or `large`.
model = MusicGen.get_pretrained('facebook/musicgen-melody')
if USE_DIFFUSION_DECODER:
    mbd = MultiBandDiffusion.get_mbd_musicgen()

Next, let us configure the generation parameters. Specifically, you can control the following:
* `use_sampling` (bool, optional): use sampling if True, else do argmax decoding. Defaults to True.
* `top_k` (int, optional): top_k used for sampling. Defaults to 250.
* `top_p` (float, optional): top_p used for sampling, when set to 0 top_k is used. Defaults to 0.0.
* `temperature` (float, optional): softmax temperature parameter. Defaults to 1.0.
* `duration` (float, optional): duration of the generated waveform. Defaults to 30.0.
* `cfg_coef` (float, optional): coefficient used for classifier free guidance. Defaults to 3.0.

When left unchanged, MusicGen will revert to its default parameters.

In [ ]:
model.set_generation_params(
    use_sampling=True,
    top_k=250,
    duration=10 # adjust this to match length of input melody prompt
)

### Melody-conditional Generation

Add the description of your remix to the 'remix_styles' array.

Then, modify the path to the base melody to load: ```torchaudio.load("../assets/your_melody.mp3")```

Finally, if you didn't already, **adjust your model generation duration parameter to match the length of the input melody**

In [ ]:
import torchaudio
from audiocraft.utils.notebook import display_audio

remix_styles=[
        '80s pop track with bassy drums and synth',
        '90s rock song with loud guitars and heavy drums',
        # '64kbps 16khz lofi hiphop summer smooth',
        # 'classical guitar solo, accoustic, flamenco',
]

melody_waveform, sr = torchaudio.load("../assets/bach.mp3")
display_audio(melody_waveform, sample_rate=32000)
melody_waveform = melody_waveform.unsqueeze(0).repeat(len(remix_styles), 1, 1)
output = model.generate_with_chroma(
    descriptions=remix_styles,
    melody_wavs=melody_waveform,
    melody_sample_rate=sr,
    progress=True, return_tokens=True
)
display_audio(output[0], sample_rate=32000)
if USE_DIFFUSION_DECODER:
    out_diffusion = mbd.tokens_to_wav(output[1])
    display_audio(out_diffusion, sample_rate=32000)